In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
%tensorflow_version 1.x
import os
import sys
# import random
# import math
# import re
# import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import urllib.request
import shutil
import zipfile
import skimage.color
import skimage.io
import skimage.transform
import imgaug

root_dir = os.path.abspath("/content/drive/My Drive/maskrcnn-nnfl/Mask_RCNN") # KUNAL'S path
#ROOT_DIR = os.path.abspath("/content/drive/.shortcut-targets-by-id/18VderTbR8Ygk4oZ9m8Upm57jX9-b5oo6/repos/Mask_RCNN") # KUNAL'S path

# mask rcnn import
sys.path.append(root_dir)
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

model_dir = os.path.join(root_dir, "logs")

sys.path.append(os.path.join(root_dir, "samples/coco/"))  # To find local version
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from pycocotools import mask as maskUtils

In [0]:
!cp -r /content/drive/My\ Drive/maskrcnn-nnfl/Mask_RCNN/samples/coco/smaller_dataset /content/

# Dataset download

In [0]:
dataType = "val"
dataYear = "2014"
dataDir = "./dataset"

# # Setup paths and file names
# if dataType == "minival" or dataType == "valminusminival":
#     imgDir = "{}/{}{}".format(dataDir, "val", dataYear)
#     imgZipFile = "{}/{}{}.zip".format(dataDir, "val", dataYear)
#     imgURL = "http://images.cocodataset.org/zips/{}{}.zip".format("val", dataYear)
# else:
#     imgDir = "{}/{}{}".format(dataDir, dataType, dataYear)
#     imgZipFile = "{}/{}{}.zip".format(dataDir, dataType, dataYear)
#     imgURL = "http://images.cocodataset.org/zips/{}{}.zip".format(dataType, dataYear)
# # print("Image paths:"); print(imgDir); print(imgZipFile); print(imgURL)

# # Create main folder if it doesn't exist yet
# if not os.path.exists(dataDir):
#     os.makedirs(dataDir)

# # Download images if not available locally
# if not os.path.exists(imgDir):
#     os.makedirs(imgDir)
#     # print("Downloading images to " + imgZipFile + " ...")
#     # with urllib.request.urlopen(imgURL) as resp, open(imgZipFile, 'wb') as out:
#     #     shutil.copyfileobj(resp, out)
#     # print("... done downloading.")
#     print("Unzipping " + imgZipFile)
#     with zipfile.ZipFile(imgZipFile, "r") as zip_ref:
#         zip_ref.extractall(dataDir)
#     print("... done unzipping")
# print("Will use images in " + imgDir)

# # Setup annotations data paths
# annDir = "{}/annotations".format(dataDir)
# if dataType == "minival":
#     annZipFile = "{}/instances_minival2014.json.zip".format(dataDir)
#     annFile = "{}/instances_minival2014.json".format(annDir)
#     annURL = "https://dl.dropboxusercontent.com/s/o43o90bna78omob/instances_minival2014.json.zip?dl=0"
#     unZipDir = annDir
# elif dataType == "valminusminival":
#     annZipFile = "{}/instances_valminusminival2014.json.zip".format(dataDir)
#     annFile = "{}/instances_valminusminival2014.json".format(annDir)
#     annURL = "https://dl.dropboxusercontent.com/s/s3tw5zcg7395368/instances_valminusminival2014.json.zip?dl=0"
#     unZipDir = annDir
# else:
#     annZipFile = "{}/annotations_trainval{}.zip".format(dataDir, dataYear)
#     annFile = "{}/instances_{}{}.json".format(annDir, dataType, dataYear)
#     annURL = "http://images.cocodataset.org/annotations/annotations_trainval{}.zip".format(dataYear)
#     unZipDir = dataDir
# print("Annotations paths:"); print(annDir); print(annFile); print(annZipFile); print(annURL)

# # Download annotations if not available locally
# print(annDir)
# if not os.path.exists(annDir):
#     os.makedirs(annDir)
# if not os.path.exists(annFile):
#     if not os.path.exists(annZipFile):
#         print("Downloading zipped annotations to " + annZipFile + " ...")
#         with urllib.request.urlopen(annURL) as resp, open(annZipFile, 'wb') as out:
#             shutil.copyfileobj(resp, out)
#         print("... done downloading.")
#     print("Unzipping " + annZipFile)
#     print(annZipFile)
#     with zipfile.ZipFile(annZipFile, "r") as zip_ref:
#         zip_ref.extractall(unZipDir)
#     print("... done unzipping")
# print("Will use annotations in " + annFile)

In [0]:
# print(len(os.listdir('./dataset/train2014')))
# print(len(os.listdir('./val2014')))
# print(os.listdir('./dataset/train2014'))
# import json
# with open('./dataset/annotations/instances_val2014.json') as f:
#   data = json.load(f)
# print(data.keys())
# print(len(data['images']))
# print(data['annotations'][0])
# !unzip -l ./dataset/val2014.zip | wc
# !zip -T ./dataset/val2014.zip
# print("here")
# !zip -T ./dataset/train2014.zip
# !unzip ./dataset/val2014.zip -d ./dataset/
# !jar xvf ./dataset/val2014.zip
# !jar --help
# !ls ./dataset/train2014 | wc

# mkdir /content/dataset
# !cd dataset
# !jar xvf /content/drive/My\ Drive/repos/Mask_RCNN/samples/coco/dataset/val2014.zip
# print(len(os.listdir('/content/val2014')))

# !cp /content/drive/My\ Drive/repos/Mask_RCNN/samples/coco/dataset/train2014.zip /content/
# !cp /content/drive/My\ Drive/repos/Mask_RCNN/samples/coco/dataset/val2014.zip /content/
# !jar xvf /content/train2014.zip
# !jar xvf /content/val2014.zip

# !cp -r /content/drive/My\ Drive/repos/Mask_RCNN/samples/coco/dataset/annotations /content/

In [0]:
# mv val2014 ./dataset/
# ls ./dataset

In [0]:
!cp -r /content/drive/My\ Drive/repos/Mask_RCNN/samples/coco/smaller_dataset /content/
# !rm -rf /content/smaller_dataset

# Loading datasets

In [0]:
# dataset_dir = './dataset'
# # subset = 'train'
# year = '2014'

# train_coco = COCO("{}/annotations/instances_{}{}.json".format(dataset_dir, 'train', year))
# val_coco = COCO("{}/annotations/instances_{}{}.json".format(dataset_dir, 'val', year))
# minival_coco = COCO("{}/annotations/instances_{}{}.json".format(dataset_dir, 'minival', year))
# valminusminival_coco = COCO("{}/annotations/instances_{}{}.json".format(dataset_dir, 'valminusminival', year))

In [0]:
# print("Train:", len(list(train_coco.imgs.keys())), "imgs")
# print("Val:", len(list(val_coco.imgs.keys())), "imgs")
# print("Minival:", len(list(minival_coco.imgs.keys())), "imgs")
# print("Valminusminival:", len(list(valminusminival_coco.imgs.keys())), "imgs")

NameError: ignored

# class CocoDataset()

In [0]:
class CocoDataset():
    def __init__(self):
        # self._image_ids = []
        self.image_ids = []
        self.image_info = []
        self.class_info = [{"source": "", "id": 0, "name": "BG"}]
        self.source_class_ids = {}

    # @property
    # def image_ids(self):
    #     return self._image_ids

    def prepare(self):
        # Build (or rebuild) everything else from the info dicts.
        self.num_classes = len(self.class_info)
        self.class_names = [",".join(ci["name"].split(",")[:1]) for ci in self.class_info]
        self.num_images = len(self.image_info)
        # self._image_ids = np.arange(self.num_images)
        self.image_ids = np.arange(self.num_images)
        self.class_ids = np.arange(self.num_classes)

        # source to id dict
        self.source_to_class_id = {"{}.{}".format(info['source'], info['id']): id
                                      for info, id in zip(self.class_info, self.class_ids)}
        # self.image_from_source_map = {"{}.{}".format(info['source'], info['id']): id
        #                               for info, id in zip(self.image_info, self.image_ids)}

        self.sources = list(set([info['source'] for info in self.class_info]))
        self.source_class_ids = {}
        for source in self.sources:
            self.source_class_ids[source] = []
            for i, info in enumerate(self.class_info):
                if i == 0 or source == info['source']:
                    self.source_class_ids[source].append(i)

    def load_coco(self, dataset_dir, subset, class_ids=None, return_coco=False):
        coco = COCO("{}/annotations/instances_{}{}.json".format(dataset_dir, subset, 2014))
        if subset == "minival" or subset == "valminusminival":
            subset = "val"
        # image_dir = "{}/{}{}".format(dataset_dir, subset, '2014')

        if class_ids is None: # all images
            class_ids = sorted(coco.getCatIds())

        if class_ids: # subset
            image_ids = []
            for id in class_ids:
                image_ids.extend(list(coco.getImgIds(catIds=[id])))
            image_ids = list(set(image_ids)) # remove dups
        else: # all images
            image_ids = list(coco.imgs.keys())

        # adding classes
        for i in class_ids:
            # self.add_class("coco", i, coco.loadCats(i)[0]["name"])
            for info in self.class_info:
                if info['source'] == "coco" and info["id"] == i:
                    return
            # class not already present
            self.class_info.append({
                "source": "coco",
                "id": i,
                "name": coco.loadCats(i)[0]["name"],
            })

        # image info
        for i in image_ids:
            image_info = {
                "id": i,
                "source": "coco",
                "path": os.path.join("{}/{}{}".format(dataset_dir, subset, '2014'), coco.imgs[i]['file_name']),
                "width": coco.imgs[i]["width"],
                "height": coco.imgs[i]["height"],
                "annotations": coco.loadAnns(coco.getAnnIds(
                    imgIds=[i], catIds=class_ids, iscrowd=None))
            }
            self.image_info.append(image_info)
            # self.add_image(
            #     "coco", image_id=i,
            #     path=os.path.join(image_dir, coco.imgs[i]['file_name']),
            #     width=coco.imgs[i]["width"],
            #     height=coco.imgs[i]["height"],
            #     annotations=coco.loadAnns(coco.getAnnIds(
                    # imgIds=[i], catIds=class_ids, iscrowd=None)))

        if return_coco:
            return coco

    # def map_source_class_id(self, source_class_id):
    #     return self.source_to_class[source_class_id]

    def load_image(self, image_id):
        image = skimage.io.imread(self.image_info[image_id]['path'])
        # convert to rgb if not
        if image.ndim != 3:
            image = skimage.color.gray2rgb(image)
        # remove alpha if there
        if image.shape[-1] == 4:
            image = image[..., :3]
        return image

    def load_mask(self, img_id):
        img = self.image_info[img_id]
        # if image_info["source"] != "coco":
        #     return super(CocoDataset, self).load_mask(image_id)

        curr_masks = []
        cls_ids = []
        anns = self.image_info[img_id]["annotations"]
        for ann in anns:
            # class_id = self.map_source_class_id(
            #     "coco.{}".format(annotation['category_id']))
            cls_id = self.source_to_class_id["coco.{}".format(ann['category_id'])]
            if cls_id:
                msk = self.annToMask(ann, img["height"],
                                   img["width"])
                if msk.max() < 1:
                    continue
                if ann['iscrowd']:
                    # -ve class id for crowd
                    cls_id *= -1
                    if msk.shape[0] != img["height"] or msk.shape[1] != img["width"]:
                        msk = np.ones([img["height"], img["width"]], dtype=bool)
                curr_masks.append(msk)
                cls_ids.append(cls_id)

        # curr masks into array
        if cls_ids:
            msk = np.stack(curr_masks, axis=2).astype(np.bool)
            cls_ids = np.array(cls_ids, dtype=np.int32)
            return msk, cls_ids
        else:
            # empty mask
            msk = np.empty([0, 0, 0])
            cls_ids = np.empty([0], np.int32)
            return msk, cls_ids
            # return super(CocoDataset, self).load_mask(image_id)

    def annToMask(self, ann, h, w):
        seg = ann['segmentation']
        if isinstance(seg, list):
            rles = maskUtils.frPyObjects(seg, h, w)
            out_rle = maskUtils.merge(rles)
        elif isinstance(seg['counts'], list):
            # uncompressed rle
            out_rle = maskUtils.frPyObjects(seg, h, w)
        else:
            out_rle = ann['segmentation']
        msk = maskUtils.decode(out_rle)
        return msk

In [13]:
dataset = '/content/smaller_dataset'
dataset_train = CocoDataset()
dataset_train.load_coco(dataset, "train")

# dataset_train.load_coco(dataset, "valminusminival", year='2014')
dataset_train.prepare()

# Validation dataset
dataset_val = CocoDataset()
val_type = "minival"
dataset_val.load_coco(dataset, val_type)
dataset_val.prepare()

# Image Augmentation
# Right/Left flip 50% of the time
augmentation = imgaug.augmenters.Fliplr(0.5)

# *** This training schedule is an example. Update to your needs ***

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [14]:
# need to remove config
class CocoConfig(Config):
  IMAGES_PER_GPU = 1
  
  #NUM_CLASSES=1+80

config = CocoConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                13
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [15]:
model = modellib.MaskRCNN(mode="training", config=CocoConfig(), model_dir=model_dir)
#model.load_weights(last_weights, by_name=True)

 
123456789
 
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

scores= Tensor("ROI/strided_slice:0", shape=(?, ?), dtype=float32)
ix= Tensor("ROI/top_anchors:1", shape=(?, 6000), dtype=int32)
scores= Tensor("ROI/ExpandDims:0", shape=(1, 6000), dtype=float32)
deltas= Tensor("ROI/ExpandDims_1:0", shape=(1, 6000, 4), dtype=float32)
pre_nms_anchors= Tensor("ROI/ExpandDims_2:0", shape=(1, 6000, 4), dtype=float32)
boxes= Tensor("ROI/ExpandDims_3:0", shape=(1, 6000, 4), dtype=float32)
boxes= Tensor("ROI/ExpandDims_4:0", shape=(1, 6000, 4), dtype=float32)
proposals= Tensor("ROI/ExpandDims_5:0", shape=(1, ?, 4), dtype=float32)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


# Training - Stage 1


In [0]:

print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=0.001,
            epochs=10,
            layers='all',
            augmentation=augmentation,
            steps_per_epoch=10)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /content/drive/My Drive/repos/Mask_RCNN/logs/coco20200520T1844/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNormalization)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNormalization)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNormalization)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNormalization)
bn2a_branch1           (BatchNormalization)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNormalization)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNormalization)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNormalization)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNormalization)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNormalization)
res2c_branch2c         (Conv2D)
bn2c_branch2c    

/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/10
10/10 [==============================] - 69s 7s/step - loss: 11.6589 - val_loss: 5.5921

Epoch 2/10
10/10 [==============================] - 16s 2s/step - loss: 14.5327 - val_loss: 5.7658
Epoch 3/10
10/10 [==============================] - 16s 2s/step - loss: 11.9209 - val_loss: 5.7750
Epoch 4/10
10/10 [==============================] - 16s 2s/step - loss: 12.5659 - val_loss: 5.7481
Epoch 5/10
10/10 [==============================] - 16s 2s/step - loss: 11.6465 - val_loss: 5.4648
Epoch 6/10
10/10 [==============================] - 16s 2s/step - loss: 11.7027 - val_loss: 5.8786
Epoch 7/10
10/10 [==============================] - 16s 2s/step - loss: 13.1471 - val_loss: 6.2325
Epoch 8/10
10/10 [==============================] - 16s 2s/step - loss: 13.0992 - val_loss: 6.5958
Epoch 9/10
10/10 [==============================] - 16s 2s/step - loss: 11.0236 - val_loss: 6.1489
Epoch 10/10
10/10 [==============================] - 16s 2s/step - loss: 11.9835 - val_loss: 6.2774


# Inference

In [0]:
# get weigths from the latest training
# Get directory names. Each directory corresponds to a model
dir_names = next(os.walk(model_dir))[1]
key = "coco" # NAME in config class
dir_names = filter(lambda f: f.startswith(key), dir_names)
dir_names = sorted(dir_names)
if not dir_names:
    import errno
    raise FileNotFoundError(
        errno.ENOENT,
        "Could not find model directory under {}".format(model_dir))
# Pick last directory
dir_name = os.path.join(model_dir, dir_names[-1])
# Find the last checkpoint
checkpoints = next(os.walk(dir_name))[2]
print(checkpoints)
checkpoints = filter(lambda f: f.startswith("mask_rcnn"), checkpoints)
checkpoints = sorted(checkpoints)
if not checkpoints:
    import errno
    raise FileNotFoundError(
        errno.ENOENT, "Could not find weight files in {}".format(dir_name))
checkpoint = os.path.join(dir_name, checkpoints[-1])
print(checkpoint)

['events.out.tfevents.1589959285.3101652c4c05', 'mask_rcnn_coco_0001.h5', 'mask_rcnn_coco_0002.h5', 'mask_rcnn_coco_0003.h5', 'mask_rcnn_coco_0004.h5', 'mask_rcnn_coco_0005.h5', 'mask_rcnn_coco_0006.h5', 'mask_rcnn_coco_0007.h5', 'mask_rcnn_coco_0008.h5', 'mask_rcnn_coco_0009.h5', 'mask_rcnn_coco_0010.h5']
/content/drive/My Drive/repos/Mask_RCNN/logs/coco20200520T0720/mask_rcnn_coco_0010.h5


In [0]:
weights = checkpoint # last weights

In [0]:
model = modellib.MaskRCNN(mode="inference", model_dir=model_dir, config=CocoConfig())
model.load_weights(weights, by_name=True) # init model again after running this
                                          # if need to start fresh training

 
qwertyuiop
 
scores= Tensor("ROI_1/strided_slice:0", shape=(?, ?), dtype=float32)
ix= Tensor("ROI_1/top_anchors:1", shape=(?, ?), dtype=int32)
scores= Tensor("ROI_1/ExpandDims:0", shape=(1, ?), dtype=float32)
deltas= Tensor("ROI_1/ExpandDims_1:0", shape=(1, ?, 4), dtype=float32)
pre_nms_anchors= Tensor("ROI_1/ExpandDims_2:0", shape=(1, ?, 4), dtype=float32)
boxes= Tensor("ROI_1/ExpandDims_3:0", shape=(1, ?, 4), dtype=float32)
boxes= Tensor("ROI_1/ExpandDims_4:0", shape=(1, ?, 4), dtype=float32)
proposals= Tensor("ROI_1/ExpandDims_5:0", shape=(1, ?, 4), dtype=float32)


Instructions for updating:
Use `tf.cast` instead.


ValueError: ignored

In [0]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
               
img_dir = '/content/smaller_dataset/val2014/'
img = skimage.io.imread(img_dir+os.listdir(img_dir)[1])

out = model.detect([img], verbose=1)

visualize.display_instances(img, out[0]['rois'], out[0]['masks'], out[0]['class_ids'], 
                            class_names, out[0]['scores'])